In [ ]:
from mhkit import dolfyn as dlfn
from mhkit.dolfyn.adp import api
from mhkit.dolfyn.rotate import api as ap
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.io
from numpy.fft import fft, ifft
from datetime import datetime
import xarray as xr
import scipy.interpolate as interp
import utm
from mhkit.river.io import d3d 
from os.path import abspath, dirname, join, normpath, relpath
import netCDF4
plt.rcParams.update({'font.size': 15}) # Set font size of plots title and labels 

In [ ]:
ds_02 = api.read('data/river/ADCP/tanana_transects_08_10_10_0_002_10-08-10_142214.PD0')
ds_03 = api.read('data/river/ADCP/tanana_transects_08_10_10_0_003_10-08-10_143335.PD0')

ds_02_03= xr.merge([ds_02,ds_03])
ds_02_03

In [ ]:
data=utm.from_latlon(ds_02_03.latitude_gps, ds_02_03.longitude_gps, 5, 'W')
latitude=data[0]
longitude=data[1]

In [ ]:
# Downloading Data
datadir = normpath(join(relpath(join('data', 'river', 'd3d'))))
filename= 'tanana50_map.nc'
d3d_data = netCDF4.Dataset(join(datadir,filename)) 

# Printing variable and description
for var in d3d_data.variables.keys():
    try: 
        d3d_data[var].long_name
    except:
        print(f'"{var}"')        
    else:
        print(f'"{var}": {d3d_data[var].long_name}')

In [ ]:
variable= 'ucx' 
var_data_df= d3d.get_all_data_points(d3d_data, variable, time_index=4)
print(var_data_df)


In [ ]:
xmin=var_data_df.x.max()
xmax=var_data_df.x.min()

ymin=var_data_df.y.max()
ymax=var_data_df.y.min()

zmin=var_data_df.waterdepth.max()
zmax=var_data_df.waterdepth.min()

In [ ]:
Lat_long=open('lat_long.txt')
lat_long=Lat_long.readlines()
Lat= lat_long[0:579]
Long= lat_long[581:1160]
Lat = np.array(Lat)
Lat=Lat.astype(float)
Long = np.array(Long)
Long=Long.astype(float)



In [ ]:
x_lat= np.tile(Lat, 100)
y_long= np.tile(Long, 100)
z_samples =np.repeat( np.linspace(zmin,zmax, num=100),579)
Point={'x': x_lat, 'y': y_long, 'waterdepth': z_samples}
points=pd.DataFrame(Point)
points

In [ ]:
turbine_x_loc= 400993.57467338
turbine_y_loc= 7161237.67096936
turbine_diameter= 0.7
N=1
y_sample = turbine_y_loc+N*turbine_diameter
x_samples = np.linspace(turbine_x_loc-60, turbine_x_loc+60, num=40)
z_samples = np.linspace(zmin,zmax, num=256)

variables= ['ucy']
sample_points = d3d.create_points(x_samples, y_sample, z_samples) 

Var_sample= d3d.variable_interpolation(d3d_data, variables, points= points)

# Creating new plot limits 
max_plot_TI=27
min_plot_TI=0


# Plotting 
plt.figure(figsize=(10,4.4))
contour_plot = plt.tricontourf(
    Var_sample.x, 
    -Var_sample.waterdepth, 
    Var_sample.ucy,

)

plt.xlabel('x (m)')
plt.ylabel('Water Depth (m)')
plt.title('Velocity in the Y direction')
cbar= plt.colorbar(contour_plot)
cbar.set_label('velocity [m/s]')

In [ ]:
sample_points

In [ ]:
np.tile([1, 2,3,4],3)

In [ ]:
plt.scatter(var_data_df.x,var_data_df.y,var_data_df.waterdepth)
plt.scatter(Lat, Long)

In [ ]:
np.size(Lat)

In [ ]:
np.size(Long)